# MNIST handwritten digits classification with linear methods

In this notebook, we'll classify handwritten digits using linear classifiers and scikit-learn.

First, the needed imports. 

In [ ]:
%matplotlib inline

import numpy as np
from sklearn import svm, datasets, __version__
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from distutils.version import LooseVersion as LV
assert(LV(__version__) >= LV("0.20")), "Version >= 0.20 of sklearn is required."

## MNIST digit data

Then we will load the MNIST data. The first time, it will download the data over the network, which can take a while.

In [ ]:
mnist = datasets.fetch_openml('mnist_784')

train_len = 60000
X = mnist['data']
y = mnist['target']

X_train, y_train = X[:train_len], y[:train_len]
X_test, y_test = X[train_len:], y[train_len:]     
     
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

The training data `X_train` is a matrix of size 60000x784, i.e., it consists of 60000 images expressed as vectors of length 784.  These vectors are in fact "flattened" 28x28 images, where each component corresponds the gray scale value of a pixel (0=black, 0.5=middle gray, 1=white, etc.).

`y_train` is a 60000-dimensional vector containing the correct classes ("0", "1", ..., "9") for each training sample.


### Reshaping a matrix

Let's looks briefly at reshaping a matrix.  Say we have a vector `A` that has a length of 9.

In [ ]:
A = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
print('A has size {} and looks like this:\n{}'.format(A.shape, A))

Now we can reshape it to a matrix of size 3x3:

In [ ]:
B = A.reshape(3,3)
print('B has size {} and looks like this:\n{}'.format(B.shape, B))

### Plotting images

Let's take a closer look at the MNIST images. Here are the first 10 training digits plotted as images together with the correct class label:

In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:].reshape(28,28), cmap="gray")
    plt.title('Class: '+str(y_train[i]))

Note that for each digit we use `reshape(28,28)` to transform the 768-size vector into a 28x28 size image matrix.

## Logistic regression

Let's start by trying logistic regression with a stochastic gradient descent algorithm.  The corresponding scikit-learn class is [LogisticRegression](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression).

### Learning

We'll actually use just the first 10,000 samples as it is rather slow. We are using the "sag" solver (which is a variant of SGD), and for one-versus-rest for doing multi-class classification.

In [ ]:
%%time

clf_lr = LogisticRegression(solver='sag', multi_class='ovr')
clf_lr.fit(X_train[:10000,:], y_train[:10000])

### Inference

In [ ]:
pred_lr = clf_lr.predict(X_test)
print('Predicted', len(pred_lr), 'digits with accuracy:', accuracy_score(y_test, pred_lr))

### Confusion matrix

We can compute the confusion matrix to see which digits get mixed the most:

In [ ]:
labels=[str(i) for i in range(10)]
print('Confusion matrix (rows: true classes; columns: predicted classes):'); print()
cm=confusion_matrix(y_test, pred_lr, labels=labels)
print(cm); print()

If we plot it as an image, we can see it more visually.  The matrix looks quite good as most image are on the diagonal, meaning they were classified correctly.

In [ ]:
plt.matshow(cm, cmap=plt.cm.gray)
plt.show()

### Accuracy, precision and recall

Classification accuracy for each class:

In [ ]:
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print("%d: %.4f" % (i,j))

Precision and recall for each class:

In [ ]:
print(classification_report(y_test, pred_lr, labels=labels))

### Failure analysis

We can also do some failure analysis.  Let's check the 10 first wrongly predicted digits.

In [ ]:
def show_failures(pred, trueclass=None, predictedclass=None, maxtoshow=10):
    errors = pred!=y_test
    print('Showing max', maxtoshow, 'first failures. '
          'The predicted class is shown first and the correct class in parenthesis.')
    ii = 0
    plt.figure(figsize=(maxtoshow, 1))
    for i in range(X_test.shape[0]):
        if ii>=maxtoshow:
            break
        if errors[i]:
            if trueclass is not None and y_test[i] != trueclass:
                continue
            if predictedclass is not None and pred[i] != predictedclass:
                continue
            plt.subplot(1, maxtoshow, ii+1)
            plt.axis('off')
            plt.imshow(X_test[i,:].reshape(28, 28), cmap="gray")
            plt.title("{} ({})".format(pred[i], y_test[i]))
            ii = ii + 1

In [ ]:
show_failures(pred_lr)

We can use `show_failures()` to inspect failures in more detail. For example:

In [ ]:
# show failures in which the true class was "5":
show_failures(pred_lr, trueclass='5')

In [ ]:
# show failures in which the prediction was "0":
show_failures(pred_lr, predictedclass='0')

In [ ]:
# show failures in which the true class was "0" and the prediction was "2":
show_failures(pred_lr, trueclass='0', predictedclass='2')

## Linear SVM

### Learning

Next we'll use linear SVM, which is similar to logistic regression, but uses a "hinge" loss instead.  Let's use the [`LinearSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) class, as it is a specialized in linear SVMs. `C` is the penalty parameter.

In [ ]:
%%time

C = 1.0
clf_lsvm = svm.LinearSVC(C=C, multi_class='ovr')
clf_lsvm.fit(X_train[:10000,:], y_train[:10000])

The training of a Linear SVM is rather fast, so it seems more data could easily be used in the training.

### Inference

As the decision boundaries are linear, prediction with linear SVMs is fast:

In [ ]:
pred_lsvm = clf_lsvm.predict(X_test)
print('Predicted', len(pred_lsvm), 'digits with accuracy:', accuracy_score(y_test, pred_lsvm))

### Confusion matrix

We can compute the confusion matrix to see which digits get mixed the most, and look at classification accuracies separately for each class:

In [ ]:
labels=[str(i) for i in range(10)]
print('Confusion matrix (rows: true classes; columns: predicted classes):'); print()
cm=confusion_matrix(y_test, pred_lsvm, labels=labels)
print(cm); print()

If we plot it as an image, we can see it more visually.  The matrix looks quite good as most image are on the diagonal, meaning they were classified correctly.

In [ ]:
plt.matshow(cm, cmap=plt.cm.gray)
plt.show()

### Accuracy, precision and recall

Classification accuracy for each class:

In [ ]:
for i,j in enumerate(cm.diagonal()/cm.sum(axis=1)): print("%d: %.4f" % (i,j))

Precision and recall for each class:

In [ ]:
print(classification_report(y_test, pred_lsvm, labels=labels))

### Failure analysis

We can also do some failure analysis.  Let's check the 10 first wrongly predicted digits.

In [ ]:
def show_failures(pred, trueclass=None, predictedclass=None, maxtoshow=10):
    errors = pred!=y_test
    print('Showing max', maxtoshow, 'first failures. '
          'The predicted class is shown first and the correct class in parenthesis.')
    ii = 0
    plt.figure(figsize=(maxtoshow, 1))
    for i in range(X_test.shape[0]):
        if ii>=maxtoshow:
            break
        if errors[i]:
            if trueclass is not None and y_test[i] != trueclass:
                continue
            if predictedclass is not None and pred[i] != predictedclass:
                continue
            plt.subplot(1, maxtoshow, ii+1)
            plt.axis('off')
            plt.imshow(X_test[i,:].reshape(28, 28), cmap="gray")
            plt.title("{} ({})".format(pred[i], y_test[i]))
            ii = ii + 1

In [ ]:
show_failures(pred_lsvm)

We can use `show_failures()` to inspect failures in more detail. For example:

In [ ]:
# show failures in which the true class was "5":
show_failures(pred_lsvm, trueclass='5')

In [ ]:
# show failures in which the prediction was "0":
show_failures(pred_lsvm, predictedclass='0')

In [ ]:
# show failures in which the true class was "0" and the prediction was "2":
show_failures(pred_lsvm, trueclass='0', predictedclass='2')

## Model tuning

Study the scikit-learn documentation of [`LogisticRegression`](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) and [`LinearSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). Experiment with different hyperparameter values.

Can you improve on the accuracy or make training faster?

Report the highest classification accuracy you manage to obtain. Also mark down the parameters you used, so others can try to reproduce your results.
